In [50]:
# We import the necessary libraries
#import sim           # library to connect with CoppeliaSim
import sympy as sp   # library for symbolic calculation
import numpy as np
from sympy import *
from sympy import cos, sin 

from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
from sympy.physics.mechanics import dynamicsymbols
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d = dynamicsymbols('theta1 theta2 d3 lc la lb theta alpha a d')




In [51]:
import coppeliasim_zmqremoteapi_client
from coppeliasim_zmqremoteapi_client import RemoteAPIClient


def connect(port=23000):
    try:
        client = RemoteAPIClient('127.0.0.1', port)
        print(f"Connected to CoppeliaSim on port {port}")
        return client
    except Exception as e:
        print(f"Could not connect to CoppeliaSim on port {port}: {e}")
        return None

client = connect()
sim = client.getObject('sim')


def setEffector(val):
    # function that triggers the end effector remotely
    # val is Int with value 0 or 1 to disable or activate the final actuator.
    scriptHandle = sim.getScriptHandle('suctionPad')
    sim.callScriptFunction('setEffector', scriptHandle, [val])
    return True

# We require the handlers for the joints, the suction cup and the suction sensor
tip = sim.getObjectHandle('suctionPadSensor')
suction = sim.getObjectHandle('suctionPad')
joint1 = sim.getObjectHandle('Joint1')
joint2 = sim.getObjectHandle('Joint2')
joint3 = sim.getObjectHandle('Joint3')

Connected to CoppeliaSim on port 23000


In [53]:
import time
import numpy as np
from sympy import cos, sin

# Constants
DISK_HEIGHT = 0.02    # Height of each disk
SAFE_HEIGHT = 0.13    # Safe height for movements
PICKUP_HEIGHT = -0.089 # Height for picking up disk from ground
QUICK_DELAY = 0.2     # Quick delay for fast movements
NORMAL_DELAY = 1.0    # Normal delay for regular movements
DROP_OFFSET = 0.005   # Extra height for dropping to avoid friction

def quick_release_and_lift(height_offset=0):
    """
    Function to drop disk from slightly higher position and quickly lift up
    Args:
        height_offset: Additional height to consider for stacked disks
    """
    # Move to a position slightly above the target
    target_height = PICKUP_HEIGHT + height_offset + DROP_OFFSET
    sim.setJointTargetPosition(joint3, target_height)
    time.sleep(NORMAL_DELAY)  # Give time to stabilize
    
    # Release and immediately start moving up
    setEffector(0)
    time.sleep(QUICK_DELAY)
    
    # Quick move to intermediate height
    sim.setJointTargetPosition(joint3, target_height + 0.02)
    time.sleep(QUICK_DELAY)
    
    # Finally move to safe height
    sim.setJointTargetPosition(joint3, SAFE_HEIGHT)
    time.sleep(NORMAL_DELAY)

# Get handles for the disks
red_disk = sim.getObjectHandle('Cylinder0')
blue_disk = sim.getObjectHandle('Cylinder')
green_disk = sim.getObjectHandle('Cylinder1')

# Get real positions of the disks from the simulator
red_pos = sim.getObjectPosition(red_disk, -1)
blue_pos = sim.getObjectPosition(blue_disk, -1)
green_pos = sim.getObjectPosition(green_disk, -1)

print("Red disk position:", red_pos)
print("Blue disk position:", blue_pos)
print("Green disk position:", green_pos)

def calculate_inverse_kinematics(x, y):
    eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x
    eq2 = 0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2) - y
    
    angle = np.arctan2(y, x)
    initial_guesses = [
        (angle, 0),
        (angle/2, angle/2),
        (0, angle),
        (0, 0),
        (np.pi/4, np.pi/4)
    ]
    
    for guess in initial_guesses:
        try:
            sol = nsolve((eq1, eq2), (theta1, theta2), guess, verify=False)
            return [float(sol[0]), float(sol[1])]
        except:
            continue
    
    raise Exception(f"Could not find solution for x={x}, y={y}")

try:
    # Calculate inverse kinematics
    print("\nCalculating inverse kinematics...")
    q_red = calculate_inverse_kinematics(red_pos[0], red_pos[1])
    q_blue = calculate_inverse_kinematics(blue_pos[0], blue_pos[1])
    q_green = calculate_inverse_kinematics(green_pos[0], green_pos[1])
    
    print("\nStarting movement sequence...")
    
    # 1. Pick up red disk and place it on blue disk
    sim.setJointTargetPosition(joint3, SAFE_HEIGHT)
    time.sleep(NORMAL_DELAY)
    
    # Position over red disk
    sim.setJointTargetPosition(joint1, q_red[0])
    sim.setJointTargetPosition(joint2, q_red[1])
    time.sleep(NORMAL_DELAY)
    
    # Lower and pick up
    sim.setJointTargetPosition(joint3, PICKUP_HEIGHT)
    time.sleep(NORMAL_DELAY)
    setEffector(1)  # Activate suction
    time.sleep(QUICK_DELAY)
    
    # Lift with disk
    sim.setJointTargetPosition(joint3, SAFE_HEIGHT)
    time.sleep(NORMAL_DELAY)
    
    # Move to blue disk position
    sim.setJointTargetPosition(joint1, q_blue[0])
    sim.setJointTargetPosition(joint2, q_blue[1])
    time.sleep(NORMAL_DELAY)
    
    # Place red disk on blue disk with quick release from higher position
    quick_release_and_lift(DISK_HEIGHT)
    
    # 2. Pick up green disk and place it on red disk
    sim.setJointTargetPosition(joint1, q_green[0])
    sim.setJointTargetPosition(joint2, q_green[1])
    time.sleep(NORMAL_DELAY)
    
    # Lower and pick up
    sim.setJointTargetPosition(joint3, PICKUP_HEIGHT)
    time.sleep(NORMAL_DELAY)
    setEffector(1)  # Activate suction
    time.sleep(QUICK_DELAY)
    
    # Lift with disk
    sim.setJointTargetPosition(joint3, SAFE_HEIGHT)
    time.sleep(NORMAL_DELAY)
    
    # Move to stack position
    sim.setJointTargetPosition(joint1, q_blue[0])
    sim.setJointTargetPosition(joint2, q_blue[1])
    time.sleep(NORMAL_DELAY)
    
    # Place green disk on red disk with quick release from higher position
    quick_release_and_lift(2 * DISK_HEIGHT)
    
    print("Sequence completed successfully!")

except Exception as e:
    print("Error during execution:", str(e))

Red disk position: [0.20985579094569745, 0.21000549855313572, 0.009999927040469429]
Blue disk position: [0.3938073243293313, -0.000872783988962302, 0.00999992713238662]
Green disk position: [0.2998510472246766, 0.10004517410478893, 0.009999927133314448]

Calculating inverse kinematics...

Starting movement sequence...
Sequence completed successfully!
